### Chapter 2

In interactive mode, Spark session is created implicitly. When you start it through a standalone application, you need to first create the SparkSession Object

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [ ]:
spark.version

The cluster of machines that Spark will use to execute tasks is managed by a cluster manager like
Spark’s standalone cluster manager, YARN, or Mesos. We then submit Spark Applications to
these cluster managers, which will grant resources to our application so that we can complete our
work

You control your Spark Application through a driver process called the SparkSession. The SparkSession instance is the way Spark executes
user-defined manipulations across the cluster. There is a one-to-one correspondence between a
SparkSession and a Spark Application. In Scala and Python, the variable is available as spark
when you start the console. 

Spark Applications consist of a driver process and a set of executor processes. The driver process
runs your main() function, sits on a node in the cluster, and is responsible for three things:
maintaining information about the Spark Application; responding to a user’s program or input;
and analyzing, distributing, and scheduling work across the executors (discussed momentarily).
The driver process is absolutely essential—it’s the heart of a Spark Application and maintains all
relevant information during the lifetime of the application. The executors are responsible for actually carrying out the work that the driver assigns them.
This means that each executor is responsible for only two things: executing code assigned to it
by the driver, and reporting the state of the computation on that executor back to the driver node

The driver and executors are simply
processes, which means that they can live on the same machine or different machines. Spark employs a cluster manager that keeps track of the resources available. The driver process is responsible for executing the driver program’s commands across
the executors to complete a given task. The executors, for the most part, will always be running Spark code. However, the driver can be
“driven” from a number of different languages through Spark’s language APIs. 

Each language API maintains the same core concepts that we described earlier. There is a
SparkSession object available to the user, which is the entrance point to running Spark code.
When using Spark from Python or R, you don’t write explicit JVM instructions; instead, you
write Python and R code that Spark translates into code that it then can run on the executor
JVMs

Executor - driver & Data - Name nodes are software pieces

When using Spark from Python or R, you don’t write explicit JVM instructions; instead, you
write Python and R code that Spark translates into code that it then can run on the executor
JVMs.

Let’s now perform the simple task of creating a range of numbers. This range of numbers is just
like a named column in a spreadsheet

In [ ]:
myRange = spark.range(1000).toDF("number")

You just ran your first Spark code! We created a DataFrame with one column containing 1,000
rows with values from 0 to 999. This range of numbers represents a distributed collection. When
run on a cluster, each part of this range of numbers exists on a different executor. This is a Spark
DataFrame.

A DF in Spark is a table which can spread on several thousand computers. The schema of a dataframe is a list which defines columns and types. You can also easily convert Pandas DF into Spark DF, etc.

A DF is then split into multiple partitions, to hand over to each executor for parallel processing. Each partition contains multiple rows and sits on one physical machine. If you have one partition, then you can only have parallelism of 1 even if you have thousands of executors. If you have one executor and thousand of partitions then still you have parallelism of one. In Dataframes you usually do not manipulate partitions manually but it is usually done through low-level APIs

Core Data structures are immutable in Spark, like tuples in Python. You instruct Spark on how to modify these through transformations, which are only acted upon if there is a action alonside

In [ ]:
divisBy2 = myRange.where("number % 2 = 0")

No output of the above command, because there is no action specified. Transformations are of two type (i) Narrow & (ii) Wide dependencies. Narrow is when each partition contribute to only one output partition and wide when each partition contribute to many partitions. The later is also referred to as shuffle as partitions change their positions in cluster. For former, Spark implements a process called pipelineing where if multiple filters are applied on a DF then they are applied in memory cache.

Lazy evaluation means waiting until the last momen to execute a graph of computation instructions i.e. a plan of transformations. THis allows optimizing the entire workflow from end to end. Predicate push down is when Spark push down a filter defined at the end of workflow to minimize data read outtime. 

In [ ]:
divisBy2.count()

Transformations allow us to build up our logical plan and actions trigger the computaiton. Three types of actions include (i) view data in console (ii) actions to collect data in the console (iii) Actions to write output data sources. 

You can view progress of a job in Spark UI available at localhost:4040

# An end-end Spark workflow

Show them the repository on Github. https://github.com/databricks/Spark-The-Definitive-Guide

In follow up time apply the same operations with other data files

we want Spark to take a best guess at what
the schema of our DataFrame should be. We also want to specify that the first row is the header
in the file, so we’ll specify that as an option, too.

In [ ]:
flightData2015 = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .csv("./../data/flight-data/csv/2015-summary.csv")

Spark peaks at only a few rows and try to infer schema but it does not have a specified number of rows. Reading data is a lazy operation. You can view outcomes of your command if you apply an action

In [ ]:
flightData2015.take(5)

In [ ]:
flightData2015.sort("count")

In [ ]:
flightData2015.sort("count").explain()

Note here that sort is a wide transformation, so the number of partitions defined are by default 200

In [ ]:
spark.conf.set("spark.sql.shuffle.partitions", "1")

Having too many partitions means too many processes & shufflig whereas too less partitions means some cores working hard and others idle

In [ ]:
flightData2015.sort("count").explain()

In [ ]:
flightData2015.sort("count").take(1)

Logical plan also keep memory of steps taken from input to output and at any point it can recreate the same results as far as the operations stay constant

In [ ]:
dataFrameWay = flightData2015\
  .groupBy("DEST_COUNTRY_NAME")\
  .count()

See here that you need to convert to a temp view. In SQL, a view is a virtual table based on the result-set of an SQL statement. A view contains rows and columns, just like a real table

In [ ]:
flightData2015.createOrReplaceTempView("flight_data_2015")

In [ ]:
sqlWay = spark.sql("""
SELECT DEST_COUNTRY_NAME, count(1)
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
""")

Each of these DataFrames (in Scala and Python) have a set of columns with an unspecified
number of rows.

In [ ]:
sqlWay.explain()

In [ ]:
dataFrameWay.explain()

Various available functionality & manipulations in Spark. Its just a row count for US - US flights

In [ ]:
spark.sql("SELECT max(count) from flight_data_2015").take(1)

In [ ]:
from pyspark.sql.functions import max
flightData2015.select(max("count")).take(1)

In [ ]:
maxSql = spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5
""")

In [ ]:
from pyspark.sql.functions import desc

In [ ]:
maxsql = flightData2015\
  .groupBy("DEST_COUNTRY_NAME")\
  .sum("count")\
  .withColumnRenamed("sum(count)", "destination_total")\
  .sort(desc("destination_total"))\
  .limit(5)\

maxsql.show()

Then applying the same operation in Dataframe

In [ ]:
flightData2015\
  .groupBy("DEST_COUNTRY_NAME")\
  .sum("count")\
  .withColumnRenamed("sum(count)", "destination_total")\
  .sort(desc("destination_total"))\
  .limit(5)\
  .explain()

This true execution plan (the one visible in explain) will differ from that shown in Figure 2-10
because of optimizations in the physical execution; however, the llustration is as good of a
starting point as any. This execution plan is a directed acyclic graph (DAG) of transformations,
each resulting in a new immutable DataFrame, on which we call an action to generate a result

The first step is to read in the data. We defined the DataFrame previously but, as a reminder,
Spark does not actually read it in until an action is called on that DataFrame or one derived from
the original DataFrame.
The second step is our grouping; technically when we call groupBy, we end up with a
RelationalGroupedDataset, which is a fancy name for a DataFrame that has a grouping
specified but needs the user to specify an aggregation before it can be queried further. We
basically specified that we’re going to be grouping by a key (or set of keys) and that now we’re
going to perform an aggregation over each one of those keys.
Therefore, the third step is to specify the aggregation. Let’s use the sum aggregation method.
This takes as input a column expression or, simply, a column name. The result of the sum
method call is a new DataFrame. You’ll see that it has a new schema but that it does know the
type of each column. It’s important to reinforce (again!) that no computation has been
performed. This is simply another transformation that we’ve expressed, and Spark is simply able
to trace our type information through it.
The fourth step is a simple renaming. We use the withColumnRenamed method that takes two
arguments, the original column name and the new column name. Of course, this doesn’t perform
computation: this is just another transformation!
The fifth step sorts the data such that if we were to take results off of the top of the DataFrame,
they would have the largest values in the destination_total column.
You likely noticed that we had to import a function to do this, the desc function. You might also
have noticed that desc does not return a string but a Column. In general, many DataFrame
methods will accept strings (as column names) or Column types or expressions. Columns and
expressions are actually the exact same thing.
Penultimately, we’ll specify a limit. This just specifies that we only want to return the first five
values in our final DataFrame instead of all the data.
The last step is our action! Now we actually begin the process of collecting the results of our
DataFrame, and Spark will give us back a list or array in the language that we’re executing. To
reinforce all of this, let’s look at the explain plan for the previous query

## Chapter 3

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [7]:
staticDataFrame = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .option("multiLine", "true")\
  .load("../data/retail-data/by-day/*.csv")

staticDataFrame.createOrReplaceTempView("retail_data")
staticSchema = staticDataFrame.schema

In [9]:
from pyspark.sql.functions import window, column, desc, col

staticDataFrame\
  .selectExpr(
    "CustomerId",
    "(UnitPrice * Quantity) as total_cost",
    "InvoiceDate")\
  .groupBy(
    col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
  .sum("total_cost")\
  .sort(desc("sum(total_cost)"))\
  .show(5)

+----------+--------------------+------------------+
|CustomerId|              window|   sum(total_cost)|
+----------+--------------------+------------------+
|   17450.0|[2011-09-20 02:00...|          71601.44|
|      null|[2011-11-14 01:00...|          55316.08|
|      null|[2011-11-07 01:00...|          42939.17|
|      null|[2011-03-29 02:00...| 33521.39999999998|
|      null|[2011-12-08 01:00...|31975.590000000007|
+----------+--------------------+------------------+
only showing top 5 rows



In [ ]:
streamingDataFrame = spark.readStream\
    .schema(staticSchema)\
    .option("maxFilesPerTrigger", 1)\
    .format("csv")\
    .option("header", "true")\
    .load("/data/retail-data/by-day/*.csv")

In [ ]:
purchaseByCustomerPerHour = streamingDataFrame\
  .selectExpr(
    "CustomerId",
    "(UnitPrice * Quantity) as total_cost",
    "InvoiceDate")\
  .groupBy(
    col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
  .sum("total_cost")

In [ ]:
purchaseByCustomerPerHour.writeStream\
    .format("memory")\
    .queryName("customer_purchases")\
    .outputMode("complete")\
    .start()

In [ ]:
spark.sql("""
  SELECT *
  FROM customer_purchases
  ORDER BY `sum(total_cost)` DESC
  """)\
  .show(5)

In [ ]:
from pyspark.sql.functions import date_format, col
preppedDataFrame = staticDataFrame\
  .na.fill(0)\
  .withColumn("day_of_week", date_format(col("InvoiceDate"), "EEEE"))\
  .coalesce(5)

In [ ]:
trainDataFrame = preppedDataFrame\
  .where("InvoiceDate < '2011-07-01'")
testDataFrame = preppedDataFrame\
  .where("InvoiceDate >= '2011-07-01'")

In [ ]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer()\
  .setInputCol("day_of_week")\
  .setOutputCol("day_of_week_index")

In [ ]:
from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder()\
  .setInputCol("day_of_week_index")\
  .setOutputCol("day_of_week_encoded")

In [ ]:
from pyspark.ml.feature import VectorAssembler

vectorAssembler = VectorAssembler()\
  .setInputCols(["UnitPrice", "Quantity", "day_of_week_encoded"])\
  .setOutputCol("features")

In [ ]:
from pyspark.ml import Pipeline

transformationPipeline = Pipeline()\
  .setStages([indexer, encoder, vectorAssembler])

In [ ]:
fittedPipeline = transformationPipeline.fit(trainDataFrame)

In [ ]:
transformedTraining = fittedPipeline.transform(trainDataFrame)

In [ ]:
from pyspark.ml.clustering import KMeans
kmeans = KMeans()\
  .setK(20)\
  .setSeed(1)

In [ ]:
kmModel = kmeans.fit(transformedTraining)

In [ ]:
transformedTest = fittedPipeline.transform(testDataFrame)

In [ ]:
from pyspark.sql import Row

spark.sparkContext.parallelize([Row(1), Row(2), Row(3)]).toDF()

## Chapter 4

In [ ]:
df = spark.range(500).toDF("number")
df.select(df["number"] + 10)

In [ ]:
spark.range(2).collect()

In [2]:
from pyspark.sql.types import *
b = ByteType()

## Chapter 5

Definitionally, a DataFrame consists of a series of records (like rows in a table), that are of type
Row, and a number of columns (like columns in a spreadsheet) that represent a computation
expression that can be performed on each individual record in the Dataset. Schemas define the
name as well as the type of data in each column. Partitioning of the DataFrame defines the
layout of the DataFrame or Dataset’s physical distribution across the cluster. The partitioning
scheme defines how that is allocated.

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

### 1. Define DF schema on read vs manual enforcing of schema

In [3]:
df = spark.read.format("json").load("./../data/flight-data/json/2015-summary.json")

In [4]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [4]:
df = spark.read.format("json").load("./../data/flight-data/json/2015-summary.json")

In [7]:
df.schema

StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,true)))

In [8]:
from pyspark.sql.types import StructField, StructType, StringType, LongType

myManualSchema = StructType([
  StructField("DEST_COUNTRY_NAME", StringType(), True),
  StructField("ORIGIN_COUNTRY_NAME", StringType(), True),
  StructField("count", LongType(), False, metadata={"hello":"world"})
])
df = spark.read.format("json").schema(myManualSchema)\
  .load("./../data/flight-data/json/2015-summary.json")

In [9]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



StructType: Represents values with the structure described by a sequence of StructFields
StructField: A field in Struct Type. metadatadict: a dict from string to simple type that can be toInternald to JSON automatically

### 2. Columns & expressions

In [10]:
from pyspark.sql.functions import col, column
col("someColumnName")
column("someColumnName")

Column<b'someColumnName'>

In [11]:
a = col("new_col")
a

Column<b'new_col'>

In [12]:
df["DEST_COUNTRY_NAME"]

Column<b'DEST_COUNTRY_NAME'>

In [13]:
df.columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count']

Expressions are operations for selection, manipulation adn removal of columns from dataframes

columns are logical constructions that simply represent a value computed on a perrecord basis by means of an expression

An expression is
a set of transformations on one or more values in a record in a DataFrame. Think of it like a
function that takes as input one or more column names, resolves them, and then potentially
applies more expressions to create a single value for each record in the dataset. Importantly, this
“single value” can actually be a complex type like a Map or Array.

In [14]:
(((col("someCol") + 5) * 200) - 6) < col("otherCol")

Column<b'((((someCol + 5) * 200) - 6) < otherCol)'>

In [15]:
from pyspark.sql.functions import expr
expr("(((someCol + 5) * 200) - 6) < otherCol")

Column<b'((((someCol + 5) * 200) - 6) < otherCol)'>

The key take away here is the underlying structure of operations in represented similarly, irrespective of whether we use SQL or DataFrames

### 3. Rows

Calling first row of a dataframe

In [16]:
df.first()

Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15)

Create rows

The key take away here is if you create a Row manually, you must specify the values in the same order as the schema of the DataFrame to which they might be appended 

In [17]:
from pyspark.sql import Row
myRow = Row("Hello", None, 1, False)

Access elements of a row

In [18]:
myRow[0]


'Hello'

In [19]:
myRow[2]

1

### 4. Data frames

One way to create dataframe is by reading a data file e.g. CSV

In [20]:
df = spark.read.format("json").load("./../data/flight-data/json/2015-summary.json")

Create temporary view

In [21]:
df.createOrReplaceTempView("dfTable")

Second way to create DF. Collecting multiple rows into a dataframe

In [22]:
from pyspark.sql import Row
from pyspark.sql.types import StructField, StructType, StringType, LongType
myManualSchema = StructType([
  StructField("some", StringType(), True),
  StructField("col", StringType(), True),
  StructField("names", LongType(), False)
])
myRow = Row("Hello", None, 1)
myDf = spark.createDataFrame([myRow , myRow], myManualSchema)
myDf.show()

+-----+----+-----+
| some| col|names|
+-----+----+-----+
|Hello|null|    1|
|Hello|null|    1|
+-----+----+-----+



You can insert multiple rows in this array here.

### 5. Select vs Expr for indexing dataframes in Spark

In [23]:
df.select("DEST_COUNTRY_NAME").show(2)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



In [24]:
df.select("DEST_COUNTRY_NAME", "ORIGIN_COUNTRY_NAME").show(2)

+-----------------+-------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|
+-----------------+-------------------+
|    United States|            Romania|
|    United States|            Croatia|
+-----------------+-------------------+
only showing top 2 rows



Notice below, that all expr, col, column seem to do the same job

In [25]:
from pyspark.sql.functions import expr, col, column

df.select(
    expr("DEST_COUNTRY_NAME"),
    col("DEST_COUNTRY_NAME"),
    column("DEST_COUNTRY_NAME"))\
  .show(2)

+-----------------+-----------------+-----------------+
|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|
+-----------------+-----------------+-----------------+
|    United States|    United States|    United States|
|    United States|    United States|    United States|
+-----------------+-----------------+-----------------+
only showing top 2 rows



The following code is expected to show compile error, due to mixing of columns and strings but it did not for some reason

In [26]:
df.select(col("DEST_COUNTRY_NAME"), "DEST_COUNTRY_NAME")

DataFrame[DEST_COUNTRY_NAME: string, DEST_COUNTRY_NAME: string]

### 6. Selectexpr 

Selecting column and changing its name simultanously

In [27]:
df.select(expr("DEST_COUNTRY_NAME AS destination")).show(2)

+-------------+
|  destination|
+-------------+
|United States|
|United States|
+-------------+
only showing top 2 rows



expr is the most flexible reference that we can use. It can refer to a plain
column or a string manipulation of a column

Selecting column, changing its name and reverting the name

In [28]:
df.select(expr("DEST_COUNTRY_NAME as destination").alias("DEST_COUNTRY_NAME"))\
  .show(2)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



Combining these together

In [29]:
df.selectExpr("DEST_COUNTRY_NAME as newColumnName", "DEST_COUNTRY_NAME").show(2)

+-------------+-----------------+
|newColumnName|DEST_COUNTRY_NAME|
+-------------+-----------------+
|United States|    United States|
|United States|    United States|
+-------------+-----------------+
only showing top 2 rows



This combination provides simple way to build up complex expression to create new dataframe. Complex queries for non-aggregating statements can be built in this way

In [30]:
df.selectExpr(
  "*", # all original columns
  "(DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withinCountry")\
  .show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



Try the following in SQL using temp view

In [ ]:
SELECT *, (DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withinCountry
FROM dfTable
LIMIT 2

Aggregating queries on whole data frames can be built using available functions

In [31]:
df.selectExpr("avg(count)", "count(distinct(DEST_COUNTRY_NAME))").show(2)

+-----------+---------------------------------+
| avg(count)|count(DISTINCT DEST_COUNTRY_NAME)|
+-----------+---------------------------------+
|1770.765625|                              132|
+-----------+---------------------------------+



Try the following SQL for the same task using tempview

In [ ]:
SELECT avg(count), count(distinct(DEST_COUNTRY_NAME)) FROM dfTable LIMIT 2

### 7. Literals & adding columns

Literals are used to explicitly pass constant values into Spark as columns

In [32]:
from pyspark.sql.functions import lit
df.select(expr("*"), lit(1).alias("One")).show(2)

+-----------------+-------------------+-----+---+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|One|
+-----------------+-------------------+-----+---+
|    United States|            Romania|   15|  1|
|    United States|            Croatia|    1|  1|
+-----------------+-------------------+-----+---+
only showing top 2 rows



IN SQL terms: SELECT *, 1 AS one FROM dfTable LIMIT 2

Similarly you can include columns in dataframes using withcolumn statement

In [33]:
df.withColumn("numberOne", lit(1)).show(2)

+-----------------+-------------------+-----+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|numberOne|
+-----------------+-------------------+-----+---------+
|    United States|            Romania|   15|        1|
|    United States|            Croatia|    1|        1|
+-----------------+-------------------+-----+---------+
only showing top 2 rows



In [48]:
df.withColumn("withinCountry", expr("ORIGIN_COUNTRY_NAME == DEST_COUNTRY_NAME"))\
  .show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



Columns rename

In [32]:
df.withColumnRenamed("DEST_COUNTRY_NAME", "dest").columns

['dest', 'ORIGIN_COUNTRY_NAME', 'count']

### 8. Reserved characters & Keywords

For select & withColumn statements you do not need to use escape characters for reserved characters e.g. dash or space. For expressions, you need o use a backtick for this purpose.

No backticks needed

In [49]:
dfWithLongColName = df.withColumn(
    "This Long Column-Name",
    expr("ORIGIN_COUNTRY_NAME"))

In [50]:
dfWithLongColName.show(2)

+-----------------+-------------------+-----+---------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|This Long Column-Name|
+-----------------+-------------------+-----+---------------------+
|    United States|            Romania|   15|              Romania|
|    United States|            Croatia|    1|              Croatia|
+-----------------+-------------------+-----+---------------------+
only showing top 2 rows



Backticks needed here

In [ ]:
dfWithLongColName.selectExpr(
    "`This Long Column-Name`",
    "`This Long Column-Name` as `new col`")\
  .show(2)

In [ ]:
dfWithLongColName.select(expr("`This Long Column-Name`")).columns

### 9. Misc

By default Spark is case insensitive but you can change it using case sensitive option

In [55]:
spark.conf.set("spark.sql.caseSensitive", False)

Dropping columns

In [ ]:
df.drop("ORIGIN_COUNTRY_NAME").columns

In [ ]:
dfWithLongColName.drop("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME")

Changing a column type

In [56]:
df.show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



Simply converting column count (type integer) to count 2 (type long) using cast statement

In [58]:
df.withColumn("count2", col("count").cast("long")).show(2)

+-----------------+-------------------+-----+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|count2|
+-----------------+-------------------+-----+------+
|    United States|            Romania|   15|    15|
|    United States|            Croatia|    1|     1|
+-----------------+-------------------+-----+------+
only showing top 2 rows



Selecting unique rows

In [ ]:
df.select("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME").distinct().count()

In [ ]:
df.select("ORIGIN_COUNTRY_NAME").distinct().count()

Random sampling

Sampling allows to specify a fraction of rows to extract from a DataFrame and whether you’d like to sample with or without replacement:

In [72]:
seed = 5
withReplacement = False
fraction = 0.5
df.sample(withReplacement, fraction, seed).show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|            Egypt|      United States|   15|
|    United States|              India|   62|
|       Costa Rica|      United States|  588|
|          Senegal|      United States|   40|
+-----------------+-------------------+-----+
only showing top 5 rows



Random split

Splitting dataframe into two by suggesting split fraction. The seed can be any random number.

In [73]:
seed = 1
dataFrames = df.randomSplit([0.25, 0.75], seed)
dataFrames[0].count() > dataFrames[1].count() # False

False

Concatenating and Appending Rows (Union)

As dataframes are immutable, you cannot add new rows into them but instead create a new DF and take a union with existing dataframe. To be able to concatenate  two DFs, you need to have (i) same schema and (ii) same number of columns

In [75]:
from pyspark.sql import Row
schema = df.schema
newRows = [
  Row("New Country", "Other Country", 5),
  Row("New Country 2", "Other Country 3", 1)
]
parallelizedRows = spark.sparkContext.parallelize(newRows)
newDF = spark.createDataFrame(parallelizedRows, schema)

In [76]:
df.union(newDF)\
  .where("count = 1")\
  .where(col("ORIGIN_COUNTRY_NAME") != "United States")\
  .show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
|    United States|          Gibraltar|    1|
|    United States|             Cyprus|    1|
|    United States|            Estonia|    1|
|    United States|          Lithuania|    1|
|    United States|           Bulgaria|    1|
|    United States|            Georgia|    1|
|    United States|            Bahrain|    1|
|    United States|   Papua New Guinea|    1|
|    United States|         Montenegro|    1|
|    United States|            Namibia|    1|
|    New Country 2|    Other Country 3|    1|
+-----------------+-------------------+-----+



Sorting Rows

For sorting rows, you can use both orderby and sort functions. Both accept expressions as well as strings and multiple columns. The default is ascending order.

In [ ]:
df.sort("count").show(5)
df.orderBy("count", "DEST_COUNTRY_NAME").show(5)
df.orderBy(col("count"), col("DEST_COUNTRY_NAME")).show(5)

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|               Malta|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|            Croatia|    1|
|       United States|          Gibraltar|    1|
|       United States|          Singapore|    1|
+--------------------+-------------------+-----+
only showing top 5 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|     Burkina Faso|      United States|    1|
|    Cote d'Ivoire|      United States|    1|
|           Cyprus|      United States|    1|
|         Djibouti|      United States|    1|
|        Indonesia|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--

To more explicitly specify sort direction, you need to use the asc and desc functions

In [ ]:
from pyspark.sql.functions import desc, asc
df.orderBy(expr("count desc")).show(2)
df.orderBy(col("count").desc(), col("DEST_COUNTRY_NAME").asc()).show(2)

An advanced tip is to use asc_nulls_first, desc_nulls_first, asc_nulls_last, or
desc_nulls_last to specify where you would like your null values to appear in an ordered
DataFrame.

In [ ]:
from pyspark.sql.functions import asc_nulls_first, desc_nulls_first

For optimization purposes, it’s sometimes advisable to sort within each partition before another
set of transformations. You can use the sortWithinPartitions method to do this:

In [ ]:
spark.read.format("json").load("./../data/flight-data/json/*-summary.json")\
  .sortWithinPartitions("count")

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

Repartition and Coalesce

Another important optimization opportunity is to partition the data according to some frequently
filtered columns, which control the physical layout of data across the cluster including the
partitioning scheme and the number of partitions.

Repartition will incur a full shuffle of the data, regardless of whether one is necessary. This
means that you should typically only repartition when the future number of partitions is greater
than your current number of partitions or when you are looking to partition by a set of columns:

In [ ]:
df.rdd.getNumPartitions()

1

In [ ]:
df.repartition(5)

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

In [ ]:
a = df.repartition(5)

In [ ]:
a.rdd.getNumPartitions()

5

If you know that you’re going to be filtering by a certain column often, it can be worth
repartitioning based on that column

In [ ]:
df.repartition(col("DEST_COUNTRY_NAME"))

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

In [ ]:
df.repartition(5, col("DEST_COUNTRY_NAME"))

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

Coalesce, on the other hand, will not incur a full shuffle and will try to combine partitions. This
operation will shuffle your data into five partitions based on the destination country name, and
then coalesce them

Coalesce reduces the number of partitions without shuffling the data across the network. It avoids a full shuffle. If it's known that the number is decreasing then the executor can safely keep data on the minimum number of partitions, only moving the data off the extra nodes, onto the nodes that we kept.

In [ ]:
df.repartition(5, col("DEST_COUNTRY_NAME")).coalesce(2)

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

Collecting Rows to the Driver

There are times when you’ll want to collect some of your data to the driver in order to manipulate it on
your local machine.

Thus far, we did not explicitly define this operation. However, we used several different methods
for doing so that are effectively all the same. collect gets all data from the entire DataFrame,
take selects the first N rows, and show prints out a number of rows nicely.

In [ ]:
collectDF = df.limit(10)
collectDF.take(5) # take works with an Integer count
collectDF.show() # this prints it out nicely
collectDF.show(5, False)
collectDF.collect()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
|    United States|          Singapore|    1|
|    United States|            Grenada|   62|
|       Costa Rica|      United States|  588|
|          Senegal|      United States|   40|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |15   |
|United States    |Croatia            |1    |
|United States    |Ireland            |344  |
|Egypt            |United States      |15   |
|United States    |India         

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Grenada', count=62),
 Row(DEST_COUNTRY_NAME='Costa Rica', ORIGIN_COUNTRY_NAME='United States', count=588),
 Row(DEST_COUNTRY_NAME='Senegal', ORIGIN_COUNTRY_NAME='United States', count=40),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]

There’s an additional way of collecting rows to the driver in order to iterate over the entire
dataset. The method to Local Iterator collects partitions to the driver as an iterator. This
method allows you to iterate over the entire dataset partition-by-partition in a serial manner:

In [ ]:
collectDF.toLocalIterator()

Any collection of data to the driver can be a very expensive operation! If you have a large dataset and
call collect, you can crash the driver

## 10. Filter rows

Filtering rows can be performed using either filter / where operations

In [66]:
df.filter(col("count") < 2).show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



In [67]:
df.where("count < 2").show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



It is also possible to sequentially chain multiple filters in a single Spark statement

In [60]:
df.where(col("count") < 2).where(col("ORIGIN_COUNTRY_NAME") != "Croatia")\
  .show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|          Singapore|    1|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



The same can be done in SQL, as follows

In [68]:
df.createOrReplaceTempView("df")

In [70]:
spark.sql("""
  SELECT *
  FROM df
  WHERE count < 2
  """)\
  .show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
|          Moldova|      United States|    1|
|            Malta|      United States|    1|
|    United States|          Gibraltar|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



Limit the number of rows to be shown. You can use both limit and show functions for this.

In [80]:
df.limit(5).show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+



In [81]:
df.orderBy(expr("count desc")).limit(6).show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|               Malta|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|            Croatia|    1|
|       United States|          Gibraltar|    1|
|       United States|          Singapore|    1|
|             Moldova|      United States|    1|
+--------------------+-------------------+-----+



## Chapter 6

### 1. Introduction

Purpose of chapter and where to find which functions. . All of these tools exist to achieve one purpose,
to transform rows of data in one format or structure to another. This might create more rows or
reduce the number of rows available. This chapter covers building
expressions, which are the bread and butter of Spark’s structured operations.

To begin, let’s read in the DataFrame that we’ll be using
for this analysis:

### 2. Data types in Spark and converting data types of Spark types

In [98]:
df = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load("./../data/retail-data/by-day/2010-12-01.csv")

df.printSchema()
df.createOrReplaceTempView("dfTable")

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



One thing to do is to always convert native types to spark types

In [101]:
from pyspark.sql.functions import lit
df.select(lit(5), lit("five"), lit(5.0))

DataFrame[5: int, five: string, 5.0: double]

### 3. Booleans

Booleans are essential when it comes to data analysis because they are the foundation for all
filtering. Boolean statements consist of four elements: and, or, true, and false. We use these
simple structures to build logical statements that evaluate to either true or false. These statements
are often used as conditional requirements for when a row of data must either pass the test
(evaluate to true) or else it will be filtered out.

In [104]:
from pyspark.sql.functions import col

df.where(col("InvoiceNo") != 536365)\
  .select("InvoiceNo", "Description")\
  .show(5, False)

+---------+-----------------------------+
|InvoiceNo|Description                  |
+---------+-----------------------------+
|536366   |HAND WARMER UNION JACK       |
|536366   |HAND WARMER RED POLKA DOT    |
|536367   |ASSORTED COLOUR BIRD ORNAMENT|
|536367   |POPPY'S PLAYHOUSE BEDROOM    |
|536367   |POPPY'S PLAYHOUSE KITCHEN    |
+---------+-----------------------------+
only showing top 5 rows



Another option—and probably the cleanest—is to specify the predicate as an expression in a
string. This is valid for Python or Scala. Note that this also gives you access to another way of
expressing “does not equal”:

In [107]:
df.where("InvoiceNo = 536365").show(5, False)

df.where("InvoiceNo <> 536365").show(5, False)


+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |2010-12-01 08:26:00|2.55     |17850.0   |United Kingdom|
|536365   |71053    |WHITE METAL LANTERN                |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |2010-12-01 08:26:00|2.75     |17850.0   |United Kingdom|
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
+---------+-----

We mentioned that you can specify Boolean expressions with multiple parts when you use and
or or. In Spark, you should always chain together and filters as a sequential filter.
The reason for this is that even if Boolean statements are expressed serially (one after the other),
Spark will flatten all of these filters into one statement and perform the filter at the same time,
creating the and statement for us. Although you can specify your statements explicitly by using
and if you like, they’re often easier to understand and to read if you specify them serially. or
statements need to be specified in the same statement:


In [109]:
from pyspark.sql.functions import instr
priceFilter = col("UnitPrice") > 600
descripFilter = instr(df.Description, "POSTAGE") >= 1
df.where(df.StockCode.isin("DOT")).where(priceFilter | descripFilter).show()

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      null|United Kingdom|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



Boolean expressions are not just reserved to filters. To filter a DataFrame, you can also just
specify a Boolean column:

In [110]:
from pyspark.sql.functions import instr
DOTCodeFilter = col("StockCode") == "DOT"
priceFilter = col("UnitPrice") > 600
descripFilter = instr(col("Description"), "POSTAGE") >= 1
df.withColumn("isExpensive", DOTCodeFilter & (priceFilter | descripFilter))\
  .where("isExpensive")\
  .select("unitPrice", "isExpensive").show(5)

+---------+-----------+
|unitPrice|isExpensive|
+---------+-----------+
|   569.77|       true|
|   607.49|       true|
+---------+-----------+



Notice how we did not need to specify our filter as an expression and how we could use a
column name without any extra work.
If you’re coming from a SQL background, all of these statements should seem quite familiar.
Indeed, all of them can be expressed as a where clause. In fact, it’s often easier to just express
filters as SQL statements than using the programmatic DataFrame interface and Spark SQL
allows us to do this without paying any performance penalty. For example, the following two
statements are equivalent:

In [113]:
df.withColumn("isExpensive", expr("NOT UnitPrice <= 250"))\
.filter("isExpensive")\
.select("Description", "UnitPrice").show(5)

+--------------+---------+
|   Description|UnitPrice|
+--------------+---------+
|DOTCOM POSTAGE|   569.77|
|DOTCOM POSTAGE|   607.49|
+--------------+---------+



In [111]:
from pyspark.sql.functions import expr

df.withColumn("isExpensive", expr("NOT UnitPrice <= 250"))\
  .where("isExpensive")\
  .select("Description", "UnitPrice").show(5)

+--------------+---------+
|   Description|UnitPrice|
+--------------+---------+
|DOTCOM POSTAGE|   569.77|
|DOTCOM POSTAGE|   607.49|
+--------------+---------+



Warning: One “gotcha” that can come up is if you’re working with null data when creating Boolean expressions.
If there is a null in your data, you’ll need to treat things a bit differently. Here’s how you can ensure
that you perform a null-safe equivalence test:
df.where(col("Description").eqNullSafe("hello")).show()


### 4. Working with numbers

When working with big data, the second most common task you will do after filtering things is
counting things. For the most part, we simply need to express our computation, and that should
be valid assuming that we’re working with numerical data types.

To fabricate a contrived example, let’s imagine that we found out that we mis-recorded the
quantity in our retail dataset and the true quantity is equal to (the current quantity * the unit
price) + 5. This will introduce our first numerical function as well as the pow function that raises
a column to the expressed power:

In [115]:
from pyspark.sql.functions import expr, pow

fabricatedQuantity = pow(col("Quantity") * col("UnitPrice"), 2) + 5
df.select(expr("CustomerId"), fabricatedQuantity.alias("realQuantity")).show(2)

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows



Notice that we were able to multiply our columns together because they were both numerical.
Naturally we can add and subtract as necessary, as well. In fact, we can do all of this as a SQL
expression, as well:

In [116]:
df.selectExpr(
  "CustomerId",
  "(POWER((Quantity * UnitPrice), 2.0) + 5) as realQuantity").show(2)

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows



Another common numerical task is rounding. If you’d like to just round to a whole number,
oftentimes you can cast the value to an integer and that will work just fine. However, Spark also
has more detailed functions for performing this explicitly and to a certain level of precision. In
the following example, we round to one decimal place:


In [ ]:
from pyspark.sql.functions import lit, round, bround

df.select(round(lit("2.5")), bround(lit("2.5"))).show(2)

Another numerical task is to compute the correlation of two columns. For example, we can see
the Pearson correlation coefficient for two columns to see if cheaper things are typically bought
in greater quantities. We can do this through a function as well as through the DataFrame
statistic methods:


In [ ]:
from pyspark.sql.functions import corr
df.stat.corr("Quantity", "UnitPrice")
df.select(corr("Quantity", "UnitPrice")).show()


Another common task is to compute summary statistics for a column or set of columns. We can
use the describe method to achieve exactly this. This will take all numeric columns and
calculate the count, mean, standard deviation, min, and max. You should use this primarily for
viewing in the console because the schema might change in the future:


In [ ]:
df.describe().show()

If you need these exact numbers, you can also perform this as an aggregation yourself by
importing the functions and applying them to the columns that you need:

In [ ]:
from pyspark.sql.functions import count, mean, stddev_pop, min, max

There are a number of statistical functions available in the StatFunctions Package (accessible
using stat as we see in the code block below). These are DataFrame methods that you can use
to calculate a variety of different things. For instance, you can calculate either exact or
approximate quantiles of your data using the approxQuantile method:


In [ ]:
colName = "UnitPrice"
quantileProbs = [0.5]
relError = 0.05
df.stat.approxQuantile("UnitPrice", quantileProbs, relError) # 2.51

You also can use this to see a cross-tabulation or frequent item pairs (be careful, this output will
be large and is omitted for this reason):


In [ ]:
df.stat.crosstab("StockCode", "Quantity").show()

In [ ]:
df.stat.freqItems(["StockCode", "Quantity"]).show()

As a last note, we can also add a unique ID to each row by using the function
monotonically_increasing_id. This function generates a unique value for each row, starting
with 0:

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id
df.select(monotonically_increasing_id()).show(2)

There are functions added with every release, so check the documentation for more methods. For
instance, there are some random data generation tools (e.g., rand(), randn()) with which you
can randomly generate data; however, there are potential determinism issues when doing so.
(You can find discussions about these challenges on the Spark mailing list.) There are also a
number of more advanced tasks like bloom filtering and sketching algorithms available in the
stat package that we mentioned (and linked to) at the beginning of this chapter. Be sure to search
the API documentation for more information and functions.

In [ ]:
from pyspark.sql.functions import initcap
df.select(initcap(col("Description"))).show()

In [ ]:
from pyspark.sql.functions import lower, upper
df.select(col("Description"),
    lower(col("Description")),
    upper(lower(col("Description")))).show(2)

In [ ]:
from pyspark.sql.functions import lit, ltrim, rtrim, rpad, lpad, trim
df.select(
    ltrim(lit("    HELLO    ")).alias("ltrim"),
    rtrim(lit("    HELLO    ")).alias("rtrim"),
    trim(lit("    HELLO    ")).alias("trim"),
    lpad(lit("HELLO"), 3, " ").alias("lp"),
    rpad(lit("HELLO"), 10, " ").alias("rp")).show(2)

In [ ]:
from pyspark.sql.functions import regexp_replace
regex_string = "BLACK|WHITE|RED|GREEN|BLUE"
df.select(
  regexp_replace(col("Description"), regex_string, "COLOR").alias("color_clean"),
  col("Description")).show(2)

In [ ]:
from pyspark.sql.functions import translate
df.select(translate(col("Description"), "LEET", "1337"),col("Description"))\
  .show(2)

In [ ]:
from pyspark.sql.functions import regexp_extract
extract_str = "(BLACK|WHITE|RED|GREEN|BLUE)"
df.select(
     regexp_extract(col("Description"), extract_str, 1).alias("color_clean"),
     col("Description")).show(2)

In [ ]:
from pyspark.sql.functions import instr
containsBlack = instr(col("Description"), "BLACK") >= 1
containsWhite = instr(col("Description"), "WHITE") >= 1
df.withColumn("hasSimpleColor", containsBlack | containsWhite)\
  .where("hasSimpleColor")\
  .select("Description").show(3, False)

In [ ]:
from pyspark.sql.functions import expr, locate
simpleColors = ["black", "white", "red", "green", "blue"]
def color_locator(column, color_string):
  return locate(color_string.upper(), column)\
          .cast("boolean")\
          .alias("is_" + color_string)
selectedColumns = [color_locator(df.Description, c) for c in simpleColors]
selectedColumns.append(expr("*")) # has to a be Column type

df.select(*selectedColumns).where(expr("is_white OR is_red"))\
  .select("Description").show(3, False)

In [ ]:
from pyspark.sql.functions import current_date, current_timestamp
dateDF = spark.range(10)\
  .withColumn("today", current_date())\
  .withColumn("now", current_timestamp())
dateDF.createOrReplaceTempView("dateTable")

In [ ]:
from pyspark.sql.functions import date_add, date_sub
dateDF.select(date_sub(col("today"), 5), date_add(col("today"), 5)).show(1)

In [ ]:
from pyspark.sql.functions import datediff, months_between, to_date
dateDF.withColumn("week_ago", date_sub(col("today"), 7))\
  .select(datediff(col("week_ago"), col("today"))).show(1)

dateDF.select(
    to_date(lit("2016-01-01")).alias("start"),
    to_date(lit("2017-05-22")).alias("end"))\
  .select(months_between(col("start"), col("end"))).show(1)

In [ ]:
from pyspark.sql.functions import to_date, lit
spark.range(5).withColumn("date", lit("2017-01-01"))\
  .select(to_date(col("date"))).show(1)

In [ ]:
from pyspark.sql.functions import to_date
dateFormat = "yyyy-dd-MM"
cleanDateDF = spark.range(1).select(
    to_date(lit("2017-12-11"), dateFormat).alias("date"),
    to_date(lit("2017-20-12"), dateFormat).alias("date2"))
cleanDateDF.createOrReplaceTempView("dateTable2")

In [ ]:
from pyspark.sql.functions import to_timestamp
cleanDateDF.select(to_timestamp(col("date"), dateFormat)).show()

In [ ]:
from pyspark.sql.functions import coalesce
df.select(coalesce(col("Description"), col("CustomerId"))).show()

In [ ]:
df.na.drop("all", subset=["StockCode", "InvoiceNo"])

In [ ]:
df.na.fill("all", subset=["StockCode", "InvoiceNo"])

In [ ]:
fill_cols_vals = {"StockCode": 5, "Description" : "No Value"}
df.na.fill(fill_cols_vals)

In [ ]:
df.na.replace([""], ["UNKNOWN"], "Description")

In [ ]:
from pyspark.sql.functions import struct
complexDF = df.select(struct("Description", "InvoiceNo").alias("complex"))
complexDF.createOrReplaceTempView("complexDF")

In [ ]:
from pyspark.sql.functions import split
df.select(split(col("Description"), " ")).show(2)

In [ ]:
df.select(split(col("Description"), " ").alias("array_col"))\
  .selectExpr("array_col[0]").show(2)

In [ ]:
from pyspark.sql.functions import size
df.select(size(split(col("Description"), " "))).show(2) # shows 5 and 3

In [ ]:
from pyspark.sql.functions import array_contains
df.select(array_contains(split(col("Description"), " "), "WHITE")).show(2)

In [ ]:
from pyspark.sql.functions import split, explode

df.withColumn("splitted", split(col("Description"), " "))\
  .withColumn("exploded", explode(col("splitted")))\
  .select("Description", "InvoiceNo", "exploded").show(2)

In [ ]:
from pyspark.sql.functions import create_map
df.select(create_map(col("Description"), col("InvoiceNo")).alias("complex_map"))\
  .show(2)

In [ ]:
df.select(map(col("Description"), col("InvoiceNo")).alias("complex_map"))\
  .selectExpr("complex_map['WHITE METAL LANTERN']").show(2)

In [ ]:
df.select(map(col("Description"), col("InvoiceNo")).alias("complex_map"))\
  .selectExpr("explode(complex_map)").show(2)

In [ ]:
jsonDF = spark.range(1).selectExpr("""
  '{"myJSONKey" : {"myJSONValue" : [1, 2, 3]}}' as jsonString""")

In [ ]:
from pyspark.sql.functions import get_json_object, json_tuple

jsonDF.select(
    get_json_object(col("jsonString"), "$.myJSONKey.myJSONValue[1]") as "column",
    json_tuple(col("jsonString"), "myJSONKey")).show(2)

In [ ]:
from pyspark.sql.functions import to_json
df.selectExpr("(InvoiceNo, Description) as myStruct")\
  .select(to_json(col("myStruct")))

In [ ]:
from pyspark.sql.functions import from_json
from pyspark.sql.types import *
parseSchema = StructType((
  StructField("InvoiceNo",StringType(),True),
  StructField("Description",StringType(),True)))
df.selectExpr("(InvoiceNo, Description) as myStruct")\
  .select(to_json(col("myStruct")).alias("newJSON"))\
  .select(from_json(col("newJSON"), parseSchema), col("newJSON")).show(2)

In [ ]:
udfExampleDF = spark.range(5).toDF("num")
def power3(double_value):
  return double_value ** 3
power3(2.0)

In [ ]:
from pyspark.sql.functions import udf
power3udf = udf(power3)

In [ ]:
from pyspark.sql.functions import col
udfExampleDF.select(power3udf(col("num"))).show(2)

In [ ]:
udfExampleDF.selectExpr("power3(num)").show(2)
# registered in Scala

In [ ]:
from pyspark.sql.types import IntegerType, DoubleType
spark.udf.register("power3py", power3, DoubleType())

In [ ]:
udfExampleDF.selectExpr("power3py(num)").show(2)
# registered via Python

## Chapter 7